In [ ]:
# import
import numpy as np
import pandas as pd
from pathlib import Path
import chardet
import matplotlib.pyplot as plt

In [ ]:
# Create path to CSV
gaming = Path("GamingStudy/Scrub_gaming_data.csv")

In [ ]:
# read in binary mode and detect encoding
# with open("../Scrub gaming data.csv", 'rb') as f:
#     result = chardet.detect(f.read())
    
# encoding = result['encoding']



In [ ]:
# Read data file with Pandas
# gaming_df = pd.read_csv(gaming, encoding = "ISO-8859-1")
gaming_df = pd.read_csv(gaming)
gaming_df

In [ ]:
gaming_df.columns

In [ ]:
clean_game = gaming_df[["S. No.","Game","Platform","Hours","earnings","whyplay","Gender","Age","Work","Degree","Residence","Playstyle","GAD_T","SWL_T","SPIN_T"]]
clean_game

In [ ]:
clean_game.to_csv("clean_game_data.csv")


In [ ]:
# Aliyu's Code Starts Here

In [ ]:
#Aliyu's code stops here

#Amy's code starts here
# Americans vs Europeans: Who spends more time gaming, and how does it compare to their satisfaction with life scores?

In [ ]:
# create a df to see only S. No. (survey number), Residence, and SWL_T (satisfaction with life total score)
residence_df = clean_game[["S. No.","Hours", "Residence","SWL_T"]]

# pull list of countries out of Residence column
residence_df["Residence"].unique()

# drop non-US and non_European countries
# coding help from Xpert Learning Assistant
# European countries reference: https://www.worldometers.info/geography/how-many-countries-in-europe/#google_vignette
patterns_to_drop = ['South Korea', 'Japan', 'Canada', 
       'Australia', 'Malaysia',
       'Turkey', 'South Africa', 'India',
       'Argentina',
       'Singapore', 'China', 'Unknown', 
       'Saudi Arabia',
       'Jordan', 'Brazil', 'Venezuela', 'Tunisia', 'Israel',
       'Qatar', 'Mexico', 'Philippines',
       'Egypt', 'New Zealand ', 
       'Algeria', 'UAE', 'Chile', 'Lebanon',
       'Thailand', 'Peru', 'Namibia', 'Uruguay', 'Kuwait', 'Bangladesh',
       'Syria', 'Colombia', 'Dominican Republic', 'Nicaragua',
       'Hong Kong', 'Vietnam', 'Kazakhstan', 'Guadeloupe',
       'Grenada', 'Trinidad & Tobago', 'Panama', 'Indonesia',
       'Puerto Rico', 'Taiwan', 'Costa Rica', 'Belize',
       'Jamaica', 'Georgia', 'Faroe Islands', 'Guatemala', 'Moldova',
       'Mongolia', 'Honduras', 'Bahrain', 'Bolivia',
       'El Salvador', 'Ecuador', 'Pakistan', 'Republic of Kosovo',
       'St Vincent', 'Brunei', 'Fiji', 'Gibraltar ', 'Palestine']

residence_df = residence_df[~residence_df["Residence"].str.contains('|'.join(patterns_to_drop))]
residence_df                                                            

- pie chart for # players US vs Euro
- bar chart for US/Euro players vs SWL score
- ANOVA analysis?

In [ ]:
# add Region col for USA and Europe (only those two)
residence_df["USA or Eur"] = np.nan

# assign USA or Eur to simplify regions
for index, country in residence_df.iterrows():
    if country["Residence"] == "USA":
        residence_df.at[index, "USA or Eur"] = "USA"
    else:
        residence_df.at[index, "USA or Eur"] = "Europe"
residence_df.head()

# total hours played (per week) by USA residents vs hours played (per week) by European residents
count_residence = residence_df["USA or Eur"].value_counts()
print(count_residence)

print("-----------------------------------------------------------------------")

usa_hours = residence_df.loc[residence_df["USA or Eur"] == "USA", "Hours"].sum()
print("The total number of hours played per week by gamers in the USA is " + str(usa_hours))

eur_hours = residence_df.loc[residence_df["USA or Eur"]== "Europe", "Hours"].sum()
print("The total number of hours played per week by gamers in Europe is " + str(eur_hours))

print("-----------------------------------------------------------------------")

# average hours played (per week) by USA residents vs average hours played (per week) by European residents
usa_hours_avg = residence_df.loc[residence_df["USA or Eur"] == "USA", "Hours"].mean()
print("The average number of hours played per week by gamers in the USA is " + str(usa_hours_avg))

eur_hours_avg = residence_df.loc[residence_df["USA or Eur"]== "Europe", "Hours"].mean()
print("The average number of hours played per week by gamers in Europe is " + str(eur_hours_avg))

print("-----------------------------------------------------------------------")

# average SWL scores for USA residents and European residents
usa_swl = residence_df.loc[residence_df["USA or Eur"] == "USA", "SWL_T"].mean()
print("The average SWL score for USA residents is " + str(usa_swl))

eur_swl = residence_df.loc[residence_df["USA or Eur"] == "Europe", "SWL_T"].mean()
print("The average SWL score for European residents is " + str(eur_swl))

In [ ]:
# Create pie chart of US players vs European players.

# format labels and colors
labels = ["European Players","USA Players"]
sizes = [6498, 4569]
colors = ["lightskyblue","red"]
explode = [0.1, 0]

# create pie chart with the above values. Add a title
plt.pie(sizes, explode =explode, labels = labels, colors = colors, autopct = "%1.1f%%", shadow = True, startangle = 90)
plt.title("Distribution of USA Players and European Players")
plt.savefig("Images/USEuroPlayersPie.png")
plt.show()

In [ ]:
# Create a scatter plot comparing SWL Scores and Hours Played (USA vs Europe)

# find mins and maxes of hours played and SWL scores
hours_max = residence_df["Hours"].max()
hours_min = residence_df["Hours"].min()

swl_max = residence_df["SWL_T"].max()
swl_min = residence_df["SWL_T"].min()

print("Max hours played: " + str(hours_max))
print("Min hours played: " + str(hours_min))
print("Max SWL score: " + str(swl_max))
print("Min SWL score: " + str(swl_min))

# Filter data based on categories
swl_usa = residence_df.loc[residence_df["USA or Eur"] == "USA", "SWL_T"]
swl_europe = residence_df.loc[residence_df["USA or Eur"] == "Europe", "SWL_T"]

# Create histograms for "USA" and "Europe" categories
plt.figure(figsize=(12, 6))

plt.hist(swl_europe, color='lightskyblue', alpha=0.7, label='Europe', bins=10)
plt.hist(swl_usa, color='red', alpha=0.7, label='USA', bins=10)

plt.title('Distribution of SWL Scores by Category')
plt.xlabel('Satisfaction of Life Score')
plt.ylabel('Frequency')
plt.legend()
plt.savefig("Images/SWLHist.png")

plt.show()

In [ ]:
# Create histograms for "USA" and "Europe" categories
# Filter data based on categories
hours_usa = residence_df.loc[residence_df["USA or Eur"] == "USA", "Hours"]
hours_europe = residence_df.loc[residence_df["USA or Eur"] == "Europe", "Hours"]

plt.figure(figsize=(12, 6))

plt.hist(hours_europe, color='lightskyblue', alpha=0.7, label='Europe', bins=10)
plt.hist(hours_usa, color='red', alpha=0.7, label='USA', bins=10)

plt.title('Distribution of Hours Played by Category')
plt.xlabel('Hours Played per Week')
plt.ylabel('Frequency')
plt.legend()
plt.savefig("Images/HoursHist.png")

plt.show()

In [ ]:
# create summary stats for US hours played vs Euro hours played

hours_df = residence_df[["USA or Eur","Hours"]]
grouped_hours = hours_df.groupby("USA or Eur")

grouped_hours = grouped_hours.describe()
grouped_hours


In [ ]:
# create summary stats for US SWL scores vs Euro SWL scores
swl_df = residence_df[["USA or Eur","SWL_T"]]
grouped_swl = swl_df.groupby("USA or Eur")

grouped_swl = grouped_swl.describe()
grouped_swl

In [ ]:
#Amy's code stops here
#Andrew's code starts here

In [ ]:
#Anna's code stops here
#Andrew's code starts here